<h3> ART SMOOTHADVERSARIAL </h3>

Provably robust deep learning via adversarially trained smoothed classifier.

Link to the paper: [https://arxiv.org/abs/1906.04584](https://arxiv.org/abs/1906.04584)

In [5]:
import tensorflow as tf
from tensorflow import keras
import os
import numpy as n
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import cv2
import numpy as np
import pandas as pd
from sklearn import metrics
import glob
import random 
import os
import pickle
import sys
import time
from datetime import datetime
import math
from typing import *

from art.utils import load_dataset, random_targets, compute_accuracy,load_cifar10
from art.estimators.certification.randomized_smoothing import TensorFlowV2RandomizedSmoothing
from keras import backend as K

<h3> Load Data </h3>

Loading the data to be used to train the model on. Using the CIFAR10 dataset here with data augmentation and preprocessing applied.

Training data: 50000
Test data: 10000
Input Shape: (32,32,3)

In [6]:
def random_crop(img, random_crop_size):
    assert img.shape[2] == 3
    height, width = img.shape[0], img.shape[1]
    dy, dx = random_crop_size
    x = np.random.randint(0, width - dx + 1)
    y = np.random.randint(0, height - dy + 1)
    return img[y:(y+dy), x:(x+dx), :]


def crop_generator(batches, crop_length):
    while True:
        batch_x, batch_y = next(batches)
        batch_crops = np.zeros((batch_x.shape[0], crop_length, crop_length, 3))
        for i in range(batch_x.shape[0]):
            batch_crops[i] = random_crop(batch_x[i], (crop_length, crop_length))
        yield (batch_crops, batch_y)

batch_size = 100

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
train_image_generator = ImageDataGenerator(rescale = 1.0/255., horizontal_flip=True)
test_image_generator = ImageDataGenerator( rescale = 1.0/255.)
train_batches = train_image_generator.flow(x_train, y_train, batch_size=batch_size, shuffle=True)
test_batches = test_image_generator.flow(x_test, y_test,batch_size=batch_size, shuffle = False)
train_crops = crop_generator(train_batches, 32)
test_crops = crop_generator(test_batches, 32)

In [8]:
num_train_samples = 50000
x_train = np.zeros((num_train_samples, 32, 32, 3), dtype=np.float32)
y_train = np.zeros((num_train_samples,1), dtype=np.int32)

for i,(data,labels) in enumerate(train_crops):
    if i == 500:
      break
    x_train[(i) * batch_size : (i+1) * batch_size, :, :, :] = data
    y_train[(i) * batch_size : (i+1) * batch_size] = labels

num_test_samples = 10000
x_test = np.zeros((num_test_samples, 32, 32, 3), dtype=np.float32)
y_test = np.zeros((num_test_samples,1), dtype=np.uint8)

for i,(data,labels) in enumerate(test_crops):
    if i == 100:
      break
    x_test[(i) * batch_size : (i+1) * batch_size, :, :, :] = data
    y_test[(i) * batch_size : (i+1) * batch_size] = labels

y_train = np.reshape(y_train,(y_train.shape[0],))

<h3> Train SmoothAdversarial Classifier </h3>

Training the smooth adversarial classifier. Building the model first using the ResNet110 architecture. Then the optimizer and scheduler functions are defined along with the Randomized Smoothing class object. The fit invocation trains the model on the training dataset.

In [9]:
_CIFAR10_MEAN = [0.4914, 0.4822, 0.4465]
_CIFAR10_STDDEV = [0.2023, 0.1994, 0.2010]

In [10]:
# Used ResNet110 architecture in TensorFlowV2 from https://github.com/gahaalt/resnets-in-tensorflow2

# MIT License

# Copyright (c) 2019 Szymon Mikler

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

def regularized_padded_conv(*args, **kwargs):
    return tf.keras.layers.Conv2D(*args, **kwargs, padding='same', kernel_regularizer=_regularizer,
                                  kernel_initializer='he_normal', use_bias=False)


def bn_relu(x):
    x = tf.keras.layers.BatchNormalization()(x)
    return tf.keras.layers.ReLU()(x)


def shortcut(x, filters, stride, mode):
    if x.shape[-1] == filters:
        return x
    elif mode == 'B':
        return regularized_padded_conv(filters, 1, strides=stride)(x)
    elif mode == 'B_original':
        x = regularized_padded_conv(filters, 1, strides=stride)(x)
        return tf.keras.layers.BatchNormalization()(x)
    elif mode == 'A':
        return tf.pad(tf.keras.layers.MaxPool2D(1, stride)(x) if stride>1 else x,
                      paddings=[(0, 0), (0, 0), (0, 0), (0, filters - x.shape[-1])])
    else:
        raise KeyError("Parameter shortcut_type not recognized!")
    

def original_block(x, filters, stride=1, **kwargs):
    c1 = regularized_padded_conv(filters, 3, strides=stride)(x)
    c2 = regularized_padded_conv(filters, 3)(bn_relu(c1))
    c2 = tf.keras.layers.BatchNormalization()(c2)
    
    mode = 'B_original' if _shortcut_type == 'B' else _shortcut_type
    x = shortcut(x, filters, stride, mode=mode)
    return tf.keras.layers.ReLU()(x + c2)
    
    
def preactivation_block(x, filters, stride=1, preact_block=False):
    flow = bn_relu(x)
    if preact_block:
        x = flow
        
    c1 = regularized_padded_conv(filters, 3, strides=stride)(flow)
    if _dropout:
        c1 = tf.keras.layers.Dropout(_dropout)(c1)
        
    c2 = regularized_padded_conv(filters, 3)(bn_relu(c1))
    x = shortcut(x, filters, stride, mode=_shortcut_type)
    return x + c2


def bootleneck_block(x, filters, stride=1, preact_block=False):
    flow = bn_relu(x)
    if preact_block:
        x = flow
         
    c1 = regularized_padded_conv(filters//_bootleneck_width, 1)(flow)
    c2 = regularized_padded_conv(filters//_bootleneck_width, 3, strides=stride)(bn_relu(c1))
    c3 = regularized_padded_conv(filters, 1)(bn_relu(c2))
    x = shortcut(x, filters, stride, mode=_shortcut_type)
    return x + c3


def group_of_blocks(x, block_type, num_blocks, filters, stride, block_idx=0):
    global _preact_shortcuts
    preact_block = True if _preact_shortcuts or block_idx == 0 else False
    
    x = block_type(x, filters, stride, preact_block=preact_block)
    for i in range(num_blocks-1):
        x = block_type(x, filters)
    return x


def Resnet(input_shape, n_classes, l2_reg=1e-4, group_sizes=(2, 2, 2), features=(16, 32, 64), strides=(1, 2, 2),
           shortcut_type='B', block_type='preactivated', first_conv={"filters": 16, "kernel_size": 3, "strides": 1},
           dropout=0, cardinality=1, bootleneck_width=4, preact_shortcuts=True):
    
    global _regularizer, _shortcut_type, _preact_projection, _dropout, _cardinality, _bootleneck_width, _preact_shortcuts
    _bootleneck_width = bootleneck_width # used in ResNeXts and bootleneck blocks
    _regularizer = tf.keras.regularizers.l2(l2_reg)
    _shortcut_type = shortcut_type # used in blocks
    _cardinality = cardinality # used in ResNeXts
    _dropout = dropout # used in Wide ResNets
    _preact_shortcuts = preact_shortcuts
    
    block_types = {'preactivated': preactivation_block,
                   'bootleneck': bootleneck_block,
                   'original': original_block}
    
    selected_block = block_types[block_type]
    inputs = tf.keras.layers.Input(shape=input_shape)
    normalizedInputs = tf.keras.layers.Normalization(mean=_CIFAR10_MEAN, variance=_CIFAR10_STDDEV)(inputs)
    flow = regularized_padded_conv(**first_conv)(normalizedInputs)
    
    if block_type == 'original':
        flow = bn_relu(flow)
    
    for block_idx, (group_size, feature, stride) in enumerate(zip(group_sizes, features, strides)):
        flow = group_of_blocks(flow,
                               block_type=selected_block,
                               num_blocks=group_size,
                               block_idx=block_idx,
                               filters=feature,
                               stride=stride)
    
    if block_type != 'original':
        flow = bn_relu(flow)
    
    flow = tf.keras.layers.GlobalAveragePooling2D()(flow)
    outputs = tf.keras.layers.Dense(n_classes, kernel_regularizer=_regularizer)(flow)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model


def load_weights_func(model, model_name):
    try: model.load_weights(os.path.join('saved_models', model_name + '.tf'))
    except tf.errors.NotFoundError: print("No weights found for this model!")
    return model

def cifar_resnet110(block_type='preactivated', shortcut_type='B', l2_reg=1e-4, load_weights=False):
    model = Resnet(input_shape=(32, 32, 3), n_classes=10, l2_reg=l2_reg, group_sizes=(18, 18, 18), features=(16, 32, 64),
                   strides=(1, 2, 2), first_conv={"filters": 16, "kernel_size": 3, "strides": 1}, shortcut_type=shortcut_type, 
                   block_type=block_type, preact_shortcuts=False)
    if load_weights: model = load_weights_func(model, 'cifar_resnet110')
    return model

In [11]:
model = cifar_resnet110('original','B',False)

In [12]:
initial_learning_rate = 0.1

boundaries = [50, 100]
values = [0.1, 0.01, 0.001]
learning_rate_fn = keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries, values)

optimizer = tf.keras.optimizers.SGD(
    learning_rate=initial_learning_rate,
    momentum=0.9,
    name='SGD',
    decay=1e-4)

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [13]:
sigma_1 = 0.25 #Noise
rs_smoothadv_classifier = TensorFlowV2RandomizedSmoothing(model=model,
                          input_shape=(32, 32, 3),
                          loss_object = loss_object,
                          nb_classes=10,
                          scale=sigma_1,
                          num_noise_vec= 8,
                          train_multi_noise= True,
                          attack_type="PGD",
                          no_grad_attack=False,
                          epsilon=1.0,
                          num_steps=10,
                          warmup=10,
                          optimizer = optimizer,
                          scheduler = learning_rate_fn
                          )

In [15]:
rs_smoothadv_classifier.fit(x_train, y_train, nb_epochs=150, batch_size=256, train_method = 'smoothadv')

<h3> Predictions for Trained Model </h3>

Predicting on the test dataset using the trained model. The accuracy for the trained model is observed.

In [ ]:
accuracy = tf.keras.metrics.Accuracy()
x_preds_rs_1 = rs_smoothadv_classifier.predict(x_test[:500])
acc_rs_1 = accuracy(tf.argmax(x_preds_rs_1, axis=1), y_test[:500])
print("\nSmoothedAdversarial Classifier, sigma=" + str(sigma_1))
print("Accuracy: {}".format(acc_rs_1))

In [21]:
# Calculate certification accuracy for a given radius
def getCertAcc(radius, pred, y_test):
    rad_list = np.linspace(0, 2.25, 201)
    cert_acc = []
    num_cert = len(radius)
    for r in rad_list:
        rad_idx = np.where(radius >= r)[0]
        y_test_subset = y_test[rad_idx]
        cert_acc.append(np.sum(pred[rad_idx] == y_test_subset) / num_cert)
    return cert_acc

In [22]:
def calculateACR(target, prediction, radius):
  tot = 0
  cnt = 0
  for i in range(0,len(prediction)):
      if(prediction[i] == target[i]):
          tot += radius[i]
      cnt += 1
  return tot/cnt

<h3>Certification on Test Images</h3>

Performing and observing the certification over all the test dataset consisting of 10000 images. The ACR (Average Certified Radius) is computed for the certification results to understand results better.

In [23]:
# no.of test images for ACR/graph (ACR inside the graph)
start_img = 500
num_img = 500
skip = 1
N = 100000

In [ ]:
prediction_1, radius_1 = rs_smoothadv_classifier.certify(x_test[(start_img-1):(start_img-1)+(num_img*skip):skip], n=N)

In [ ]:
acr = calculateACR(target=np.array(y_test[(start_img-1):(start_img-1)+(num_img*skip):skip]), prediction= np.array(prediction_1), radius = np.array(radius_1))
print("ACR for Smooth Adversarial Classifier: ", acr)

In [ ]:
rad_list = np.linspace(0, 2.25, 201)
plt.plot(rad_list, getCertAcc(radius_1, prediction_1, np.array(y_test)), 'r-', label='smoothed, $\sigma=$' + str(sigma_1))
plt.xlabel('l2 radius')
plt.ylabel('Certified Accuracy')
plt.legend()
plt.title('Average Certified Radius plot: ACR {}'.format(acr))
plt.show()